## Task 2: Use SageMaker Clarify
	
In this lab, you use Amazon SageMaker Clarify to detect bias in pre-training data and post-training models and access explainability reports.


### Task 2.1: Environment setup

Install packages and dependencies.

In [1]:
#install-dependencies

import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import sagemaker
import sys
import time
import zipfile

from pandas.core.computation.check import NUMEXPR_INSTALLED
from IPython.display import display
from IPython.display import Image
from sagemaker import clarify
from sagemaker import Session
from sagemaker.inputs import TrainingInput
from sagemaker.s3 import S3Uploader
from sklearn.model_selection import train_test_split
from time import gmtime, strftime

s3_client = boto3.client("s3")
session = Session()
bucket = session.default_bucket()
prefix = 'sagemaker/lab_8'
role = sagemaker.get_execution_role()

/opt/conda/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


[04/20/25 13:11:29] INFO     Created S3 bucket: sagemaker-us-west-2-591057661366                     ]8;id=312674;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=893772;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#723\723]8;;\

Note: If you see the error in the output \"ERROR: pip's dependency resolver does not currently take into account all the packages that are installed'. You can ignore the package dependency error and proceed with the next steps.

Next, import, split, and upload the dataset.

In [2]:
#prepare-dataset

lab_test_data = pd.read_csv('adult_data_processed.csv')

# Split the dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)
test_data.to_csv('test_data.csv', index=False, header=False)

# Upload the Dataset to S3
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))
test_path = S3Uploader.upload('test_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')
test_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

/opt/conda/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Now, train the XGBoost model. You use this trained model for the SageMaker Clarify ModelConfig. The training takes approximately 3-4 minutes to complete.

In [ ]:
#train-model

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)
# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker_session
)
# Set the hyperparameters
xgb.set_hyperparameters(
    max_depth=5, 
    eta=0.2, 
    gamma=4, 
    min_child_weight=6,
    subsample=0.8, 
    verbosity=1, 
    objective='binary:logistic', 
    num_round=800
)

# Train the model
xgb.fit(
    inputs = data_inputs
) 

[04/20/25 13:12:57] INFO     Ignoring unnecessary instance type: None.                            ]8;id=929111;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=134274;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=738961;file:///opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=761241;file:///opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=778268;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=63943;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-04-20-13-12-57-398                                             

2025-04-20 13:13:01 Starting - Starting the training job...
2025-04-20 13:13:15 Starting - Preparing the instances for training.

### Task 2.2: Set the SageMaker Clarify job and access the bias report

To use [SageMaker Clarify](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html), you first create a model from your training job. Then you set up the necessary configurations and run the SageMaker Clarify job on your trained model. In this task, you complete the following:

- Create the model.
- Enable SageMaker Clarify.
- Run the bias report.
- Access the report.

### Task 2.3: Create the model

Create a model from the training job to use with SageMaker Clarify.

In [ ]:
#create-clarify-model

model_name = "lab-8-clarify-model"
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
session.create_model(model_name, role, container_def)

### Task 2.4: Enable SageMaker Clarify

Now, to begin configuration, enable SageMaker Clarify.

In [ ]:
#enable-clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge", 
    sagemaker_session=session
)

With SageMaker Clarify, you can detect potential pre-training bias in your data and post-training bias in your models through various metrics. Refer to [metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html) for more information about configuring the clarify analysis.

Configure the **DataConfig** object to communicate data I/O information to SageMaker Clarify. Here, you specify the location of the input dataset, the location for the output, the income (**label**) column, the header names, and the dataset type.

In [ ]:
#define-data-config

bias_report_output_path = "s3://{}/{}/clarify-bias".format(bucket, prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_path,
    s3_output_path=bias_report_output_path,
    label="income",
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)

Configure the **ModelConfig** object to communicate information about your trained model. Set the model name and a temporary dedicated endpoint to avoid additional traffic to your production model (**instance_type** and **instance_count**). Also set an **accept_type** to denote the endpoint response payload format, and a **content_type** to denote the payload format of request to the endpoint.

In [ ]:
#define-model-config

model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

Configure the **probability_threshold** of the **ModelPredictedLabelConfig** to convert the probability of samples to binary labels for bias analysis. Prediction above the threshold is interpreted as label value **1** and below or equal as label value **0**.

In [ ]:
#define-model-predicted-label-config

predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

Configure the **BiasConfig** to denote the sensitive columns (**facets**), potential sensitive features (**facet_values_or_threshold**), and favorable outcomes (**label_values_or_threshold**).

You can denote both categorical and continuous data for **facet_values_or_threshold** and **label_values_or_threshold**. The **sex** and **age** features are categorical.

The goal of this model is to determine if an individual makes 50,000 USD or more, with the positive outcome being favorable. Use **BiasConfig** to provide information on which columns contain the facets with the sensitive group of **Sex**, what the sensitive features might be using **facet_values_or_threshold**, and what the desirable outcomes are using **label_values_or_threshold**. Group by **age** to see if there are any differences depending on the individual's age.

In [ ]:
#define-bias-config

bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="sex", facet_values_or_threshold=[0], group_name="age"
)

### Task 2.5: Run the bias report

Create the bias report using your configurations for pre-training and post-training analysis. This step takes approximately 15–20 minutes. While the bias report is being generated, you can discover how SageMaker Clarify computes [Fairness Measures for Machine Learning in Finance](./Fairness.Measures.for.Machine.Learning.in.Finance.pdf).

In [ ]:
#run-bias-report

clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

### Task 2.6: Access the bias report

In SageMaker Studio, you can view the results in the S3 bucket. A bias report of the metrics is available when SageMaker Clarify finishes running the bias job.

1. Wait for SageMaker Clarify to finish running the bias job.

2. In the left navigation pane, choose the **Object Storage Browser** icon.

3. Choose the bucket that has **sagemaker** in its name.

4. Open the **sagemaker** folder.

5. Open the **lab_8** folder.

6. Open the **Clarify-bias** folder.

7. Open **report.ipynb**.

8. Choose **Python 3 (ipykernel)**.

9. Choose **Select**.

Each bias metric has detailed explanations with examples that you can explore by selecting each value. 

10. When you are finished exploring the bias metrics, continue the next task.

### Task 2.7: Access explainability reports

In addition to using a bias report, you can use SageMaker Clarify to analyze the local explanations of individual predictions. Create a report to review explainability results for the predictions produced by SageMaker Clarify and analyze key metrics from the report. In this task, you complete the following:

1. Define a Shapley Values (SHAP) configuration.
2. Run the explainability report.
3. Access the explainability report.

### Task 2.8: Define a SHAP configuration

You can run an explainability report to get an explanation of why a model made a specific prediction. SageMaker Clarify uses SHAP to generate a report on the model's reasoning for its decisions. Refer to [SHAP Baselines for Explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-shap-baselines.html) for more information about SHAP baselines.

The SHAP metrics that are configured are as follows:
- **baseline**: A list of rows, or an Amazon Simple Storage Service (Amazon S3) object URI, to be used as the baseline dataset in the Kernel SHAP algorithm.
- **num_samples**: Number of samples to be used in the Kernel SHAP algorithm. This number determines the size of the generated synthetic dataset to compute the SHAP values.
- **agg_method: mean_abs**: Mean of absolute SHAP values for all instances.
- **save_local_shap_values**: Boolean value to indicate if local SHAP values are to be saved in the output location.

Refer to [SHAP metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html) for more information about the metrics used.

In [ ]:
#configure-shap

testing_data, clarify_data = train_test_split(test_data, test_size =0.005)
clarify_data = clarify_data.drop(columns=["income"])
clarify_data.to_csv('clarify_data.csv', index=False, header=False)
clarify_path = S3Uploader.upload('clarify_data.csv', 's3://{}/{}'.format(bucket, prefix))

shap_config = clarify.SHAPConfig(
    baseline=clarify_path,
    num_samples=15,
    agg_method="mean_abs",
    save_local_shap_values=True,
)

explainability_output_path = "s3://{}/{}/clarify-explainability".format(bucket, prefix)
explainability_data_config = clarify.DataConfig(
    s3_data_input_path=clarify_path,
    s3_output_path=explainability_output_path,
    headers=clarify_data.columns.to_list(),
    dataset_type="text/csv",
)

### Task 2.9: Run the explainability report

Create the explainability report using your configurations. This step takes approximately 10–15 minutes. While the explainability report is being generated, you can follow along with the job status in SageMaker Studio by continuing with the next task.

Refer to [Amazon AI Fairness and Explainability Whitepaper](./Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf) for more information about the SageMaker Clarify explainability process.

In [ ]:
#run-explainability-report

clarify_processor.run_explainability(
    data_config=explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
)

### Task 2.10: Access the explainability report

As with the bias report, you can review the explainability report When the explainability job is finished is finished running.

1. Wait for explainability report job to finish running.

2. In the left navigation pane, navigate back to the **Object Storage Browser**.

3. Navigate to the **sagemaker/lab_8** folder.

6. Open the **clarify-explainability** folder.

7. Open **report.ipynb**.

8. Choose **Python 3 (ipykernel)**.

9. Choose **Select**.

What features have the highest and lowest importance? Are there any results that are surprising?

### Conclusion

Congratulations! You have used SageMaker Clarify to create bias and explainability reports that you can use to develop more trustworthy and compliant models. In the next lab, you deploy your models and run inference. You continue working with your model in the next lab.

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file.
- Return to the lab session and continue with the **Conclusion**.

## Task 2: Use SageMaker Clarify
	
In this lab, you use Amazon SageMaker Clarify to detect bias in pre-training data and post-training models and access explainability reports.


### Task 2.1: Environment setup

Install packages and dependencies.

In [ ]:
#install-dependencies

import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import sagemaker
import sys
import time
import zipfile

from pandas.core.computation.check import NUMEXPR_INSTALLED
from IPython.display import display
from IPython.display import Image
from sagemaker import clarify
from sagemaker import Session
from sagemaker.inputs import TrainingInput
from sagemaker.s3 import S3Uploader
from sklearn.model_selection import train_test_split
from time import gmtime, strftime

s3_client = boto3.client("s3")
session = Session()
bucket = session.default_bucket()
prefix = 'sagemaker/lab_8'
role = sagemaker.get_execution_role()

Note: If you see the error in the output \"ERROR: pip's dependency resolver does not currently take into account all the packages that are installed'. You can ignore the package dependency error and proceed with the next steps.

Next, import, split, and upload the dataset.

In [ ]:
#prepare-dataset

lab_test_data = pd.read_csv('adult_data_processed.csv')

# Split the dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)
test_data.to_csv('test_data.csv', index=False, header=False)

# Upload the Dataset to S3
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))
test_path = S3Uploader.upload('test_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')
test_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

Now, train the XGBoost model. You use this trained model for the SageMaker Clarify ModelConfig. The training takes approximately 3-4 minutes to complete.

In [ ]:
#train-model

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)
# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker_session
)
# Set the hyperparameters
xgb.set_hyperparameters(
    max_depth=5, 
    eta=0.2, 
    gamma=4, 
    min_child_weight=6,
    subsample=0.8, 
    verbosity=1, 
    objective='binary:logistic', 
    num_round=800
)

# Train the model
xgb.fit(
    inputs = data_inputs
) 

### Task 2.2: Set the SageMaker Clarify job and access the bias report

To use [SageMaker Clarify](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html), you first create a model from your training job. Then you set up the necessary configurations and run the SageMaker Clarify job on your trained model. In this task, you complete the following:

- Create the model.
- Enable SageMaker Clarify.
- Run the bias report.
- Access the report.

### Task 2.3: Create the model

Create a model from the training job to use with SageMaker Clarify.

In [ ]:
#create-clarify-model

model_name = "lab-8-clarify-model"
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
session.create_model(model_name, role, container_def)

### Task 2.4: Enable SageMaker Clarify

Now, to begin configuration, enable SageMaker Clarify.

In [ ]:
#enable-clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge", 
    sagemaker_session=session
)

With SageMaker Clarify, you can detect potential pre-training bias in your data and post-training bias in your models through various metrics. Refer to [metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html) for more information about configuring the clarify analysis.

Configure the **DataConfig** object to communicate data I/O information to SageMaker Clarify. Here, you specify the location of the input dataset, the location for the output, the income (**label**) column, the header names, and the dataset type.

In [ ]:
#define-data-config

bias_report_output_path = "s3://{}/{}/clarify-bias".format(bucket, prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_path,
    s3_output_path=bias_report_output_path,
    label="income",
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)

Configure the **ModelConfig** object to communicate information about your trained model. Set the model name and a temporary dedicated endpoint to avoid additional traffic to your production model (**instance_type** and **instance_count**). Also set an **accept_type** to denote the endpoint response payload format, and a **content_type** to denote the payload format of request to the endpoint.

In [ ]:
#define-model-config

model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

Configure the **probability_threshold** of the **ModelPredictedLabelConfig** to convert the probability of samples to binary labels for bias analysis. Prediction above the threshold is interpreted as label value **1** and below or equal as label value **0**.

In [ ]:
#define-model-predicted-label-config

predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

Configure the **BiasConfig** to denote the sensitive columns (**facets**), potential sensitive features (**facet_values_or_threshold**), and favorable outcomes (**label_values_or_threshold**).

You can denote both categorical and continuous data for **facet_values_or_threshold** and **label_values_or_threshold**. The **sex** and **age** features are categorical.

The goal of this model is to determine if an individual makes 50,000 USD or more, with the positive outcome being favorable. Use **BiasConfig** to provide information on which columns contain the facets with the sensitive group of **Sex**, what the sensitive features might be using **facet_values_or_threshold**, and what the desirable outcomes are using **label_values_or_threshold**. Group by **age** to see if there are any differences depending on the individual's age.

In [ ]:
#define-bias-config

bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="sex", facet_values_or_threshold=[0], group_name="age"
)

### Task 2.5: Run the bias report

Create the bias report using your configurations for pre-training and post-training analysis. This step takes approximately 15–20 minutes. While the bias report is being generated, you can discover how SageMaker Clarify computes [Fairness Measures for Machine Learning in Finance](./Fairness.Measures.for.Machine.Learning.in.Finance.pdf).

In [ ]:
#run-bias-report

clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

### Task 2.6: Access the bias report

In SageMaker Studio, you can view the results in the S3 bucket. A bias report of the metrics is available when SageMaker Clarify finishes running the bias job.

1. Wait for SageMaker Clarify to finish running the bias job.

2. In the left navigation pane, choose the **Object Storage Browser** icon.

3. Choose the bucket that has **sagemaker** in its name.

4. Open the **sagemaker** folder.

5. Open the **lab_8** folder.

6. Open the **Clarify-bias** folder.

7. Open **report.ipynb**.

8. Choose **Python 3 (ipykernel)**.

9. Choose **Select**.

Each bias metric has detailed explanations with examples that you can explore by selecting each value. 

10. When you are finished exploring the bias metrics, continue the next task.

### Task 2.7: Access explainability reports

In addition to using a bias report, you can use SageMaker Clarify to analyze the local explanations of individual predictions. Create a report to review explainability results for the predictions produced by SageMaker Clarify and analyze key metrics from the report. In this task, you complete the following:

1. Define a Shapley Values (SHAP) configuration.
2. Run the explainability report.
3. Access the explainability report.

### Task 2.8: Define a SHAP configuration

You can run an explainability report to get an explanation of why a model made a specific prediction. SageMaker Clarify uses SHAP to generate a report on the model's reasoning for its decisions. Refer to [SHAP Baselines for Explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-shap-baselines.html) for more information about SHAP baselines.

The SHAP metrics that are configured are as follows:
- **baseline**: A list of rows, or an Amazon Simple Storage Service (Amazon S3) object URI, to be used as the baseline dataset in the Kernel SHAP algorithm.
- **num_samples**: Number of samples to be used in the Kernel SHAP algorithm. This number determines the size of the generated synthetic dataset to compute the SHAP values.
- **agg_method: mean_abs**: Mean of absolute SHAP values for all instances.
- **save_local_shap_values**: Boolean value to indicate if local SHAP values are to be saved in the output location.

Refer to [SHAP metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html) for more information about the metrics used.

In [ ]:
#configure-shap

testing_data, clarify_data = train_test_split(test_data, test_size =0.005)
clarify_data = clarify_data.drop(columns=["income"])
clarify_data.to_csv('clarify_data.csv', index=False, header=False)
clarify_path = S3Uploader.upload('clarify_data.csv', 's3://{}/{}'.format(bucket, prefix))

shap_config = clarify.SHAPConfig(
    baseline=clarify_path,
    num_samples=15,
    agg_method="mean_abs",
    save_local_shap_values=True,
)

explainability_output_path = "s3://{}/{}/clarify-explainability".format(bucket, prefix)
explainability_data_config = clarify.DataConfig(
    s3_data_input_path=clarify_path,
    s3_output_path=explainability_output_path,
    headers=clarify_data.columns.to_list(),
    dataset_type="text/csv",
)

### Task 2.9: Run the explainability report

Create the explainability report using your configurations. This step takes approximately 10–15 minutes. While the explainability report is being generated, you can follow along with the job status in SageMaker Studio by continuing with the next task.

Refer to [Amazon AI Fairness and Explainability Whitepaper](./Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf) for more information about the SageMaker Clarify explainability process.

In [ ]:
#run-explainability-report

clarify_processor.run_explainability(
    data_config=explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
)

### Task 2.10: Access the explainability report

As with the bias report, you can review the explainability report When the explainability job is finished is finished running.

1. Wait for explainability report job to finish running.

2. In the left navigation pane, navigate back to the **Object Storage Browser**.

3. Navigate to the **sagemaker/lab_8** folder.

6. Open the **clarify-explainability** folder.

7. Open **report.ipynb**.

8. Choose **Python 3 (ipykernel)**.

9. Choose **Select**.

What features have the highest and lowest importance? Are there any results that are surprising?

### Conclusion

Congratulations! You have used SageMaker Clarify to create bias and explainability reports that you can use to develop more trustworthy and compliant models. In the next lab, you deploy your models and run inference. You continue working with your model in the next lab.

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file.
- Return to the lab session and continue with the **Conclusion**.